<font color='green'> 
**Youtube - Aladdin Persson Kanalı - TensorFlow 2.0 Beginner Tutorials serisi**
    
TensorFlow Tutorial 6 - RNNs, GRUs, LSTMs and Bidirectionality - Aladdin Persson anlattı.
</font>

**Video:** [TensorFlow Tutorial 6 - RNNs, GRUs, LSTMs and Bidirectionality](https://www.youtube.com/watch?v=Ogvd787uJO8&list=PLhhyoLH6IjfxVOdVC1P1L5z5azs0XjMsb&index=6)

### İçindekiler

**Loading Dataset**

**Preprocessing Dataset**

**Creating a Recurrent Neural Network (RNN)**

**Creating a Gated Recurrent Unit (GRU)**

**Creating a Long Short Term Memory Network (LSTM)**

**Bidirectional Long-Short Term Memory (Bidirectional LSTM)**

### Kaynaklar

* RNN, LSTM ve GRU modellerine teorik olarak hızlıca bakmak için [Illustrated Guide to LSTM’s and GRU’s: A step by step explanation](https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21#7029) ve [Bi-LSTM](https://medium.com/@raghavaggarwal0089/bi-lstm-bc3d68da8bd0) yazılarını okuyabilirsin.


* Andrew NG'nin [DeepLearning Andrew Ng Course 5 Sequence Models](https://www.youtube.com/playlist?list=PL1F3ABbhcqa3BBWo170U4Ev2wfsF7FN8l) videolarına bakabilirsin. 

### <font color="blue"> Giriş</font>

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### 1. Loading Dataset

In [3]:
from tensorflow.keras.datasets import mnist

In [4]:
(x_train, y_train),(x_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [5]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


### 2. Preprocessing Dataset

In [6]:
x_train = x_train.astype("float32") / 255.0

In [7]:
x_test = x_test.astype("float32") / 255.0

### 3. Creating a Recurrent Neural Network (RNN)

28 x 28 pixele sahip resimlerimiz var. Bu resimleri RNN'e, GRU'ya veya LSTM'e gönderdiğimizde nasıl çalışacak buna bakacağız. Her seferinde resmin bir rowunu modele sokacağız. Önce 1. row, sonra 2. row vs. Normalde image'leri işlemek için sequence modeller kullanılmaz convolutional neural networkler kullanılır. Burada sadece tensorflowda RNN, GRU ve LSTM'in nasıl implement edildiğini görselleştirmek için bu data setini kullandık. 

##### <font color="green"> `model = keras.Sequential()` şeklinde belirttikten sonra `model.add()` ile tek tek layerları ekliyoruz. </font>

In [8]:
model = keras.Sequential()

model.add(keras.Input(shape=(None, 28))) 
model.add(layers.SimpleRNN(512, return_sequences=True, activation='relu'))
model.add(layers.SimpleRNN(512, activation='relu'))
model.add(layers.Dense(10))

* `model.add(keras.Input(shape=(None, 28))) ` içerisinde shape'in ilk boyutunu 'None' olarak belirttik. Çünkü spesifik bir time stepimiz yok. İkinci boyutu 28 olarak belirtik. Çünkü Her time stepte 28 pixelimiz var. Bu casede 28 time stepimiz var. İlk dimensionı belirtmedik. 

* `layers.SimpleRNN(512, return_sequence=True, activation='relu')` SimpleRNN: basic RNN demek. 512 nod olsun dedik. `return_sequences=True` her time stepten output dönmesini sağlıyor. Bu şekilde birden fazla rnn katmanını üst üste istiflemiş oluyoruz. Bu RNN'in outputu 512 nodlu bir sequences olacak. `activation='relu'` ile de aktivasyon fonksiyonumuzu belirledik. Eğer hiçbir şey belirtmezsen default olarak 'tanh' alıyor.

- İkinci kez RNN katmanı ekledik. Burada `return_sequences = True` yapmadık. Çünkü her time stepten geçecek ve RNN'in son çıktısında dense layer alacağız ve 10 nodlu bir outputumuz olacak.

In [9]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, None, 512)         276992    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 512)               524800    
_________________________________________________________________
dense (Dense)                (None, 10)                5130      
Total params: 806,922
Trainable params: 806,922
Non-trainable params: 0
_________________________________________________________________
None


**simple_rnn (SimpleRNN) - (None, None, 512)** -> İlk RNN için 512 output nodumuz var, ikinci None her bir time stepi temsil ediyor. İki tane None olmasının sebebi bir tanesi batchler için, bir tanesi hidden stateler için.

**simple_rnn_1 (SimpleRNN) - (None, 512)** -> Bunda `return_sequences=True` dememiştik. Buradaki None batchleri ifade ediyor. 512 nod tüm inputları geçtikten sonraki son hidden stateteki 512 nodu ifade ediyor. 

**dense (Dense) - (None, 10)** -> En sonunda ise 10 nodluk bir layerımız var. 

##### <font color="green"> `model.compile()` içerisinde ağımızın eğitim bölümünü nasıl yapılandıracağımızı anlatıyoruz kerasa. </font>

In [11]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(lr=0.001),
    metrics=["accuracy"]
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


##### <font color="green"> `model.fit()` ile yapılandırdığımız modeli somut olarak eğitiyoruz. </font>

In [12]:
model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)

Epoch 1/10
938/938 - 68s - loss: 0.3073 - accuracy: 0.9049
Epoch 2/10
938/938 - 64s - loss: 0.1398 - accuracy: 0.9623
Epoch 3/10
938/938 - 64s - loss: 0.1339 - accuracy: 0.9641
Epoch 4/10
938/938 - 64s - loss: 0.1034 - accuracy: 0.9720
Epoch 5/10
938/938 - 64s - loss: 0.0950 - accuracy: 0.9740
Epoch 6/10
938/938 - 64s - loss: 0.0831 - accuracy: 0.9773
Epoch 7/10
938/938 - 64s - loss: 0.0818 - accuracy: 0.9780
Epoch 8/10
938/938 - 64s - loss: 0.0773 - accuracy: 0.9790
Epoch 9/10
938/938 - 64s - loss: 0.0715 - accuracy: 0.9802
Epoch 10/10
938/938 - 64s - loss: 0.0631 - accuracy: 0.9826


##### <font color="green"> `model.evaluate()` ile test setimizi gönderip modelimizin başarısını değerlendiriyoruz. </font>

In [13]:
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

313/313 - 3s - loss: 0.0990 - accuracy: 0.9725


[0.09900984168052673, 0.9725000262260437]

Training sette 0.98 accuracye, test sette 0.97 accuracye sahibiz. 

### 4. Creating a Gated Recurrent Unit (GRU)

##### <font color="green"> `model = keras.Sequential()` şeklinde belirttikten sonra `model.add()` ile tek tek layerları ekliyoruz. </font>

In [14]:
model = keras.Sequential()

model.add(keras.Input(shape=(None, 28))) 
model.add(layers.GRU(256, return_sequences=True, activation='tanh'))
model.add(layers.GRU(256, activation='tanh'))
model.add(layers.Dense(10))

In [15]:
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, None, 256)         219648    
_________________________________________________________________
gru_1 (GRU)                  (None, 256)               394752    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2570      
Total params: 616,970
Trainable params: 616,970
Non-trainable params: 0
_________________________________________________________________
None


##### <font color="green"> `model.compile()` içerisinde ağımızın eğitim bölümünü nasıl yapılandıracağımızı anlatıyoruz kerasa. </font>

In [16]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(lr=0.001),
    metrics=["accuracy"]
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


##### <font color="green"> `model.fit()` ile yapılandırdığımız modeli somut olarak eğitiyoruz. </font>

In [17]:
model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)

Epoch 1/10
938/938 - 21s - loss: 0.2603 - accuracy: 0.9144
Epoch 2/10
938/938 - 15s - loss: 0.0653 - accuracy: 0.9803
Epoch 3/10
938/938 - 15s - loss: 0.0457 - accuracy: 0.9862
Epoch 4/10
938/938 - 15s - loss: 0.0366 - accuracy: 0.9889
Epoch 5/10
938/938 - 15s - loss: 0.0286 - accuracy: 0.9911
Epoch 6/10
938/938 - 15s - loss: 0.0257 - accuracy: 0.9918
Epoch 7/10
938/938 - 15s - loss: 0.0188 - accuracy: 0.9939
Epoch 8/10
938/938 - 15s - loss: 0.0187 - accuracy: 0.9941
Epoch 9/10
938/938 - 15s - loss: 0.0161 - accuracy: 0.9948
Epoch 10/10
938/938 - 15s - loss: 0.0160 - accuracy: 0.9949


##### <font color="green"> `model.evaluate()` ile test setimizi gönderip modelimizin başarısını değerlendiriyoruz. </font>

In [18]:
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

313/313 - 2s - loss: 0.0424 - accuracy: 0.9877


[0.04235471412539482, 0.9876999855041504]

Parametreler farklı olduğu için bir önceki modelle karşılaştırmak doğru olmaz. Burada sadece nasıl kullandığımızı görüyoruz.

### 5. Creating a Long Short Term Memory Network (LSTM)

LSTM'ler ve GRU'lar performans açısından aynılar. LSTM'ler bir miktar daha iyi olabilir GRU'lardan. 

##### <font color="green"> `model = keras.Sequential()` şeklinde belirttikten sonra `model.add()` ile tek tek layerları ekliyoruz. </font>

In [8]:
model = keras.Sequential()

model.add(keras.Input(shape=(None, 28))) 
model.add(layers.LSTM(256, return_sequences=True, activation='tanh'))
model.add(layers.LSTM(256, activation='tanh'))
model.add(layers.Dense(10))

In [9]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, None, 256)         291840    
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense (Dense)                (None, 10)                2570      
Total params: 819,722
Trainable params: 819,722
Non-trainable params: 0
_________________________________________________________________
None


##### <font color="green"> `model.compile()` içerisinde ağımızın eğitim bölümünü nasıl yapılandıracağımızı anlatıyoruz kerasa. </font>

In [21]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(lr=0.001),
    metrics=["accuracy"]
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


##### <font color="green"> `model.fit()` ile yapılandırdığımız modeli somut olarak eğitiyoruz. </font>

In [24]:
model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)

Epoch 1/10
938/938 - 18s - loss: 0.0168 - accuracy: 0.9946
Epoch 2/10
938/938 - 18s - loss: 0.0152 - accuracy: 0.9954
Epoch 3/10
938/938 - 18s - loss: 0.0127 - accuracy: 0.9960
Epoch 4/10
938/938 - 18s - loss: 0.0130 - accuracy: 0.9960
Epoch 5/10
938/938 - 18s - loss: 0.0130 - accuracy: 0.9961
Epoch 6/10
938/938 - 18s - loss: 0.0116 - accuracy: 0.9965
Epoch 7/10
938/938 - 18s - loss: 0.0087 - accuracy: 0.9971
Epoch 8/10
938/938 - 18s - loss: 0.0095 - accuracy: 0.9969
Epoch 9/10
938/938 - 18s - loss: 0.0105 - accuracy: 0.9967
Epoch 10/10
938/938 - 18s - loss: 0.0058 - accuracy: 0.9983


##### <font color="green"> `model.evaluate()` ile test setimizi gönderip modelimizin başarısını değerlendiriyoruz. </font>

In [23]:
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

313/313 - 3s - loss: 0.0278 - accuracy: 0.9919


[0.027801280841231346, 0.9919000267982483]

### 6. Bidirectional Long-Short Term Memory (Bidirectional LSTM)

Tek yönlü LSTM kullanmak yerine çift yönlü LSTM kullanacağız. 

##### <font color="green"> `model = keras.Sequential()` şeklinde belirttikten sonra `model.add()` ile tek tek layerları ekliyoruz. </font>

In [31]:
model = keras.Sequential()

model.add(keras.Input(shape=(None, 28))) 
model.add(layers.Bidirectional(layers.LSTM(256, return_sequences=True, activation='tanh')))
model.add(layers.LSTM(256, activation='tanh'))
model.add(layers.Dense(10))

In [32]:
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, None, 512)         583680    
_________________________________________________________________
lstm_7 (LSTM)                (None, 256)               787456    
_________________________________________________________________
dense_5 (Dense)              (None, 10)                2570      
Total params: 1,373,706
Trainable params: 1,373,706
Non-trainable params: 0
_________________________________________________________________
None


İlk LSTM layerında `model.add(layers.Bidirectional(layers.LSTM(256, return_sequences=True, activation='tanh')))` eklediğimizde (None, None, 512) -> 256 nod yerine 512 nod oldu. Her hidden state için nod sayısını 256 nod olarak belirledik fakat çift yönlü olduğu için nod sayısının 2 katı kadar output oluyor. 

İkincisini de Bidirectional LSTM yapıyoruz.

In [10]:
model = keras.Sequential()

model.add(keras.Input(shape=(None, 28))) 
model.add(layers.Bidirectional(layers.LSTM(256, return_sequences=True, activation='tanh')))
model.add(layers.Bidirectional(layers.LSTM(256, activation='tanh')))
model.add(layers.Dense(10))

In [11]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, None, 512)         583680    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               1574912   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 2,163,722
Trainable params: 2,163,722
Non-trainable params: 0
_________________________________________________________________
None


##### <font color="green"> `model.compile()` içerisinde ağımızın eğitim bölümünü nasıl yapılandıracağımızı anlatıyoruz kerasa. </font>

In [12]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(lr=0.001),
    metrics=["accuracy"]
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


##### <font color="green"> `model.fit()` ile yapılandırdığımız modeli somut olarak eğitiyoruz. </font>

In [13]:
model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)

Epoch 1/10
938/938 - 49s - loss: 0.2649 - accuracy: 0.9132
Epoch 2/10
938/938 - 38s - loss: 0.0742 - accuracy: 0.9769
Epoch 3/10
938/938 - 38s - loss: 0.0541 - accuracy: 0.9830
Epoch 4/10
938/938 - 38s - loss: 0.0416 - accuracy: 0.9868
Epoch 5/10
938/938 - 38s - loss: 0.0358 - accuracy: 0.9886
Epoch 6/10
938/938 - 38s - loss: 0.0280 - accuracy: 0.9909
Epoch 7/10
938/938 - 38s - loss: 0.0247 - accuracy: 0.9922
Epoch 8/10
938/938 - 38s - loss: 0.0208 - accuracy: 0.9934
Epoch 9/10
938/938 - 38s - loss: 0.0184 - accuracy: 0.9939
Epoch 10/10
938/938 - 38s - loss: 0.0169 - accuracy: 0.9945


##### <font color="green"> `model.evaluate()` ile test setimizi gönderip modelimizin başarısını değerlendiriyoruz. </font>

In [14]:
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

313/313 - 6s - loss: 0.0371 - accuracy: 0.9896


[0.03711550682783127, 0.9896000027656555]

Performans one directional lstm ile neredeyse aynı. 